<a href="https://colab.research.google.com/github/Xeesto/UEP/blob/main/Systemy_Rekomendacyjne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [8]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [10]:
# Liczba filmów
num_movies = len(movies)
print(f"Liczba filmów w bazie: {num_movies}")

Liczba filmów w bazie: 9742


In [11]:
# Liczba ocen
num_ratings = len(ratings)
print(f"Liczba ocen w bazie: {num_ratings}")

Liczba ocen w bazie: 100836


In [12]:
# Liczba użytkowników
num_users = ratings['userId'].nunique()
print(f"Liczba użytkowników w bazie: {num_users}")


Liczba użytkowników w bazie: 610


In [13]:
# Średnia liczba ocen użytkownika
avg_ratings_per_user = ratings.groupby('userId')['rating'].count().mean()
print(f"Średnia liczba ocen użytkownika: {avg_ratings_per_user:.2f}")

Średnia liczba ocen użytkownika: 165.30


In [14]:
# Średnia liczba ocen na film
avg_ratings_per_movie = ratings.groupby('movieId')['rating'].count().mean()
print(f"Średnia liczba ocen na film: {avg_ratings_per_movie:.2f}")

Średnia liczba ocen na film: 10.37


In [15]:
# Rozkład ocen
rating_counts = ratings['rating'].value_counts().sort_index()
print("Rozkład ocen:")
print(rating_counts)

Rozkład ocen:
rating
0.5     1370
1.0     2811
1.5     1791
2.0     7551
2.5     5550
3.0    20047
3.5    13136
4.0    26818
4.5     8551
5.0    13211
Name: count, dtype: int64


In [17]:
movie_titles = dict(zip(movies['movieId'], movies['title']))

In [26]:
# Wyświetlenie słownika
# print("Słownik filmów:")
# print(movie_titles)

In [23]:
movie_id = 13                                       # ID filmu do wpisania
title = movie_titles.get(movie_id)

if title:
    print(f"Film o ID {movie_id} to: {title}")
else:
    print(f"Nie znaleziono filmu o ID: {movie_id}")

Film o ID 13 to: Balto (1995)
